In [1]:
from linearalgebra import Vector

In [2]:
def num_differences(v1: Vector, v2: Vector) -> int:
    assert len(v1) == len(v2)
    return len([x1 for x1, x2 in zip(v1, v2) if x1 != x2])

In [3]:
from typing import List
from linearalgebra import vector_mean

In [4]:
def cluster_means(k: int,
                  inputs: List[Vector],
                  assignments: List[int]) -> List[Vector]:
                  clusters = [[] for i in range(k)]
                  for input, assignment in zip(inputs, assignments):
                      clusters[assignment].append(input)
                   
                  return [vector_mean(cluster) if cluster else random.choice(inputs) for cluseter in clusters]